# Projeto ETL

## Realizando a extração dos dados:

In [ ]:
import pandas, pandera

In [ ]:
dataframe = pandas.read_csv('ocorrencia2.csv', sep=";", parse_dates=['ocorrencia_dia'], dayfirst = True)  # o separador que veio foi ;, por isso, essa opção.
# a parse_dates converte do tipo objeto para data e o dayfirst indica que o primeiro parâmetro é dia e não o mês, pq foi br a fonte
print(dataframe)

In [ ]:
# dataframe.dtypes  # o dia está como objeto,mas isso foi arrumado lá no parametro de abertura do df

# extração do mês da data
mes = dataframe.ocorrencia_dia.dt.month
print(mes)


## Validando os dados

In [ ]:
dataframe.head(10) # imprime as primeiras 10 linhas

In [ ]:
# tipo de dado:
dataframe.dtypes  # object são textos

In [ ]:
# tratando os dados com pandera:
schema = pandera.DataFrameSchema(
    columns = {
        "codigo":pandera.Column(pandera.Int, required=False),
        "codigo_ocorrencia":pandera.Column(pandera.Int),
        "codigo_ocorrencia2":pandera.Column(pandera.Int),
        "ocorrencia_classificacao":pandera.Column(pandera.String),
        "ocorrencia_cidade":pandera.Column(pandera.String),
        "ocorrencia_uf":pandera.Column(pandera.String, pandera.Check.str_length(2, 2), nullable=True), #minimos e máximos
        "ocorrencia_aerodromo":pandera.Column(pandera.String, nullable=True),
        "ocorrencia_dia":pandera.Column(pandera.DateTime),
        "ocorrencia_hora":pandera.Column(pandera.String, pandera.Check.str_matches(r"^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$"),nullable=True),  # filtra horas maiores que 24
        "total_recomendacoes":pandera.Column(pandera.Int),

    }
)

In [ ]:
schema.validate(dataframe)  # se a linha rodar, significa q o dataframe foi validade mediante a coluna selecionada em schema, ou seja, existe
# ou seja, o schema valida os dados gerando uma excessão caso surja uma divergencia

# Limpeza dos dados

In [ ]:
dataframe.head()

In [159]:
# localizando algum dado, por exemplo, belém, na linha 1
dataframe.loc[1, 'ocorrencia_cidade'] # linha 1 , ocorrencia cidade

KeyError: 1

In [ ]:
# pegar a linha toda:
dataframe.loc[1]  # isso é um label, não uma lista

In [ ]:
# pegando em um intervalo:
dataframe.loc[1:3]

In [ ]:
dataframe.loc[[10, 40]] # pegar a linha 10 e a linha 40

In [ ]:
# pegar só a coluna:
dataframe.loc[:,'ocorrencia_cidade']


In [ ]:
# alterar um índice do dataframe
# basta firma-se em uma linha que possui códigos únicos
contador = 0
lista = []
for codigos in dataframe.loc[:, 'codigo_ocorrencia']:
    if codigos not in lista:
        lista.append(codigos)
    else:
        contador += 1
print(contador) # o resultado é zero, comprovando entao que dá para usá-lo


In [ ]:
# o código acima pode ser substituído por:
dataframe.codigo_ocorrencia.is_unique  # se retornar true, ok

In [ ]:
# como é único, ele pode ser o índice para caso se deseje alterar algum valor:
dataframe.set_index('codigo_ocorrencia', inplace=True)

In [ ]:
dataframe.head()

In [ ]:
# testando uma localizacao no codigo_ocorrencia como índice
dataframe.loc[40324]

In [ ]:
# para voltar ao índice original
#dataframe.reset_index(drop=True, inplace=True)
#dataframe.head(5)

In [ ]:
# passar a coluna # ocorrencia_aerodromo
for linhas in dataframe.loc[:, 'ocorrencia_latitude']:
    if linhas == '***':
        linhas.replace('***', ' ')

In [ ]:
dataframe.loc[0, 'ocorrencia_latitude'] = ''

In [ ]:
dataframe

In [ ]:
# criar nova coluna
dataframe['nova'] = dataframe.ocorrencia_longitude # espécie de backup

In [ ]:
dataframe

In [ ]:
#if dataframe.loc[4, 'ocorrencia_cidade'] == 'PELOTAS':
#    dataframe.loc[4, 'ocorrencia_classificacao'] = 'GRAVE'

In [ ]:
dataframe

In [ ]:
# MÉTODO MAIS SIMPLES DE FAZER A TAREFA ACIMA:
dataframe.loc[dataframe.ocorrencia_uf == 'RJ', ['ocorrencia_classificacao']] = 'MUITO GRAVE'

In [ ]:
dataframe

In [ ]:
# imprimir todos os locais de Minas Gerais:
dataframe.loc[dataframe.ocorrencia_uf == 'MG']

In [ ]:
# limpando os dados
dataframe.loc[dataframe.ocorrencia_aerodromo == '****', ['ocorrencia_aerodromo']] = pandas.NA


In [ ]:
dataframe.loc[:, 'ocorrencia_aerodromo']

In [ ]:
# forma mais fácil
dataframe.replace(['**', '###', '****','*****', 'NULL'], pandas.NA, inplace=True)

In [ ]:
# somar por coluna os dados faltantes: NA
dataframe.isna().sum()

In [ ]:
# substituir todos os valores NA:
dataframe.fillna(0, inplace=True)  # inplace é pra alterar msm
dataframe.isna().sum()

In [ ]:
# alterar apenas um valor 
dataframe.fillna(value={'total_recomendacoes': 10}) # so na coluna 10, onde tiver na

In [ ]:
# para excluir uma coluna: dataframe.drop(['nome'],axis=1,inplace=True)
# se for excluir os NA com dataframe.dropna(), exclui a linha toda e nao só a célula
dataframe.drop(['codigo_ocorrencia1'],axis=1)

In [ ]:
dataframe

In [ ]:
#dataframe.drop(labels=0)

In [ ]:
# excluir coluna específica do NA
#dataframe.dropna(subset=['nomedacoluna'])
dataframe.drop_duplicates() #exclui duplicados 

In [ ]:
dataframe.drop_duplicates(inplace=True)

# Transformação de dados
 

In [ ]:
#schema.validate(dataframe)

In [ ]:
dataframe.iloc[1]

In [ ]:
# localizar dados que têm valores nulos
dataframe.loc[dataframe.ocorrencia_dia.isnull()]
# contar 
dataframe.count() # se ficar padronizado, nao há valores nulos

In [ ]:
j = 0
for i in dataframe.total_recomendacoes: # ver quaais tiveram mais de 10 recomendações
    if i > 10:
        j += 1
print(j)

In [ ]:
# jeito mais preciso
filtro = dataframe.total_recomendacoes > 10
dataframe.loc[filtro] # pode-se colocar vírgula aqui e pegar as ocorrencias de cidade etc

In [ ]:
# ver um tipo de ocorrencia e pegar a coluna cidade
filtro = dataframe.ocorrencia_classificacao == 'MUITO GRAVE'
dataframe.loc[filtro, 'ocorrencia_cidade']

In [ ]:
# localizar duas informações distintas:
filtro1 = dataframe.ocorrencia_classificacao == "INCIDENTE GRAVE"
filtro2 = dataframe.ocorrencia_uf == "SP"
dataframe.loc[filtro1 & filtro2]



In [ ]:
# outra forma de fazer isso:
filtro = dataframe.ocorrencia_classificacao.isin(["INCIDENTE GRAVE", "INCIDENTE"]) # ou é incidente grave ou é incidente
filtro2 = dataframe.ocorrencia_uf == "SP"
dataframe.loc[filtro & filtro2]

In [ ]:
# pegar apenas parte de um dado em uma coluna/linha
inci = dataframe.ocorrencia_classificacao.str[0:4]
inci

In [53]:
# buscar por conter palavra
busca = dataframe.ocorrencia_cidade.str[0] == "M" # procurar qual começa com a letra M, se fosse termina str[-1]
dataframe.loc[busca]

,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,...,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,"ocorrencia_saida_pista,,",nova
codigo_ocorrencia,,,,,,,,,,,,,,,,,,,,,
39916,39916.0,39916.0,39916.0,39916.0,INCIDENTE GRAVE,-3.0505555556,-60.0508333333,MANAUS,AM,BRASIL,...,19:35:00,SIM,FINALIZADA,IG-171/CENIPA/2013,SIM,2013-12-30,2.0,1.0,"SIM,,",-60.0508333333
40168,40168.0,40168.0,40168.0,40168.0,INCIDENTE,0,0,MONTES CLAROS,MG,BRASIL,...,01:34:00,***,0,0,NÃO,0,0.0,1.0,"NÃO,,",0
40170,40170.0,40170.0,40170.0,40170.0,INCIDENTE,0,0,MANAUS,AM,BRASIL,...,11:00:00,***,0,0,NÃO,0,0.0,1.0,"NÃO,,",0
40332,40332.0,40332.0,40332.0,40332.0,MUITO GRAVE,0,0,MACAÉ,RJ,BRASIL,...,21:58:00,SIM,FINALIZADA,0,NÃO,0,0.0,1.0,"NÃO,,",0
40258,40258.0,40258.0,40258.0,40258.0,INCIDENTE,0,0,MONTE ALEGRE,PA,BRASIL,...,18:10:00,SIM,FINALIZADA,0,NÃO,0,0.0,1.0,"NÃO,,",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80160,80160.0,80160.0,80160.0,80160.0,INCIDENTE,-3.041111111111,-60.05055555555,MANAUS,AM,BRASIL,...,22:30:00,SIM,FINALIZADA,***,NÃO,0,0.0,1.0,"NÃO,,",-60.05055555555
80222,80222.0,80222.0,80222.0,80222.0,INCIDENTE,-13.09416,-42.55250,MACAÚBAS,BA,BRASIL,...,23:49:00,SIM,FINALIZADA,***,NÃO,0,0.0,1.0,"NÃO,,",-42.55250
80205,80205.0,80205.0,80205.0,80205.0,INCIDENTE,-3.041111111111,-60.05055555555,MANAUS,AM,BRASIL,...,11:15:00,SIM,FINALIZADA,***,NÃO,0,0.0,1.0,"NÃO,,",-60.05055555555


In [58]:
# outra forma de encontrá-lo é com o método contains
#busca1 = dataframe.ocorrencia_classificacao.str.contains('A|NA| NAN') # A OU NA
#dataframe.loc[busca1]

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [88]:
for i in dataframe.divulgacao_dia_publicacao:
    if i == 0:
        pass
    else:
        print(f"ANO: {i[0:4]}")
        print(f"MÊS: {i[5:7]}")
        print(f"DIA: {i[8:]}")
        print()
    

ANO: 2019
MÊS: 10
DIA: 28

ANO: 2011
MÊS: 07
DIA: 21

ANO: 2011
MÊS: 06
DIA: 30

ANO: 2012
MÊS: 11
DIA: 19

ANO: 2011
MÊS: 06
DIA: 30

ANO: 2016
MÊS: 03
DIA: 18

ANO: 2017
MÊS: 02
DIA: 08

ANO: 2010
MÊS: 12
DIA: 06

ANO: 2011
MÊS: 05
DIA: 18

ANO: 2011
MÊS: 07
DIA: 21

ANO: 2012
MÊS: 11
DIA: 19

ANO: 2011
MÊS: 07
DIA: 21

ANO: 2014
MÊS: 10
DIA: 13

ANO: 2011
MÊS: 06
DIA: 30

ANO: 2018
MÊS: 05
DIA: 17

ANO: 2011
MÊS: 05
DIA: 18

ANO: 2011
MÊS: 08
DIA: 29

ANO: 2011
MÊS: 07
DIA: 21

ANO: 2011
MÊS: 07
DIA: 21

ANO: 2011
MÊS: 07
DIA: 21

ANO: 2016
MÊS: 04
DIA: 22

ANO: 2013
MÊS: 12
DIA: 12

ANO: 2012
MÊS: 03
DIA: 22

ANO: 2011
MÊS: 08
DIA: 01

ANO: 2018
MÊS: 12
DIA: 04

ANO: 2013
MÊS: 12
DIA: 30

ANO: 2013
MÊS: 11
DIA: 12

ANO: 2013
MÊS: 12
DIA: 30

ANO: 2016
MÊS: 04
DIA: 07

ANO: 2011
MÊS: 12
DIA: 21

ANO: 2017
MÊS: 11
DIA: 30

ANO: 2011
MÊS: 08
DIA: 01

ANO: 2011
MÊS: 07
DIA: 21

ANO: 2011
MÊS: 08
DIA: 01

ANO: 2011
MÊS: 08
DIA: 29

ANO: 2011
MÊS: 09
DIA: 23

ANO: 2018
MÊS: 12
DIA: 04

A

DIA: 18

ANO: 2014
MÊS: 06
DIA: 02

ANO: 2014
MÊS: 06
DIA: 16

ANO: 2014
MÊS: 03
DIA: 17

ANO: 2014
MÊS: 02
DIA: 27

ANO: 2014
MÊS: 04
DIA: 16

ANO: 2014
MÊS: 03
DIA: 17

ANO: 2014
MÊS: 04
DIA: 28

ANO: 2014
MÊS: 04
DIA: 24

ANO: 2015
MÊS: 06
DIA: 15

ANO: 2014
MÊS: 04
DIA: 07

ANO: 2016
MÊS: 04
DIA: 22

ANO: 2014
MÊS: 05
DIA: 29

ANO: 2014
MÊS: 06
DIA: 02

ANO: 2015
MÊS: 02
DIA: 09

ANO: 2014
MÊS: 04
DIA: 22

ANO: 2020
MÊS: 07
DIA: 24

ANO: 2015
MÊS: 03
DIA: 20

ANO: 2014
MÊS: 03
DIA: 17

ANO: 2014
MÊS: 06
DIA: 16

ANO: 2014
MÊS: 06
DIA: 16

ANO: 2014
MÊS: 03
DIA: 17

ANO: 2019
MÊS: 06
DIA: 28

ANO: 2014
MÊS: 04
DIA: 28

ANO: 2015
MÊS: 03
DIA: 06

ANO: 2014
MÊS: 06
DIA: 16

ANO: 2014
MÊS: 08
DIA: 15

ANO: 2019
MÊS: 04
DIA: 08

ANO: 2014
MÊS: 06
DIA: 16

ANO: 2016
MÊS: 07
DIA: 22

ANO: 2014
MÊS: 06
DIA: 16

ANO: 2017
MÊS: 08
DIA: 01

ANO: 2018
MÊS: 05
DIA: 10

ANO: 2015
MÊS: 07
DIA: 21

ANO: 2015
MÊS: 02
DIA: 02

ANO: 2014
MÊS: 03
DIA: 28

ANO: 2014
MÊS: 06
DIA: 16

ANO: 2018
MÊS: 09
D

ANO: 2018
MÊS: 09
DIA: 24

ANO: 2016
MÊS: 08
DIA: 26

ANO: 2016
MÊS: 10
DIA: 14

ANO: 2015
MÊS: 10
DIA: 19

ANO: 2015
MÊS: 11
DIA: 16

ANO: 2015
MÊS: 10
DIA: 19

ANO: 2019
MÊS: 03
DIA: 11

ANO: 2015
MÊS: 11
DIA: 16

ANO: 2017
MÊS: 08
DIA: 01

ANO: 2018
MÊS: 12
DIA: 19

ANO: 2015
MÊS: 10
DIA: 19

ANO: 2016
MÊS: 06
DIA: 23

ANO: 2016
MÊS: 06
DIA: 03

ANO: 2016
MÊS: 07
DIA: 07

ANO: 2017
MÊS: 04
DIA: 04

ANO: 2016
MÊS: 06
DIA: 16

ANO: 2020
MÊS: 03
DIA: 11

ANO: 2015
MÊS: 11
DIA: 16

ANO: 2015
MÊS: 11
DIA: 16

ANO: 2015
MÊS: 11
DIA: 15

ANO: 2018
MÊS: 12
DIA: 19

ANO: 2017
MÊS: 06
DIA: 22

ANO: 2016
MÊS: 02
DIA: 15

ANO: 2016
MÊS: 04
DIA: 22

ANO: 2017
MÊS: 10
DIA: 27

ANO: 2016
MÊS: 06
DIA: 16

ANO: 2016
MÊS: 02
DIA: 15

ANO: 2015
MÊS: 11
DIA: 16

ANO: 2020
MÊS: 03
DIA: 11

ANO: 2016
MÊS: 06
DIA: 16

ANO: 2016
MÊS: 10
DIA: 28

ANO: 2016
MÊS: 06
DIA: 23

ANO: 2016
MÊS: 04
DIA: 22

ANO: 2015
MÊS: 11
DIA: 16

ANO: 2015
MÊS: 11
DIA: 16

ANO: 2015
MÊS: 11
DIA: 16

ANO: 2016
MÊS: 06
DIA: 23

A


ANO: 2019
MÊS: 10
DIA: 28

ANO: 2019
MÊS: 09
DIA: 03

ANO: 2021
MÊS: 07
DIA: 21

ANO: 2019
MÊS: 10
DIA: 28

ANO: 2019
MÊS: 10
DIA: 28

ANO: 2020
MÊS: 03
DIA: 11

ANO: 2020
MÊS: 10
DIA: 29

ANO: 2019
MÊS: 09
DIA: 03

ANO: 2021
MÊS: 03
DIA: 29

ANO: 2019
MÊS: 08
DIA: 06

ANO: 2021
MÊS: 07
DIA: 08

ANO: 2021
MÊS: 07
DIA: 21

ANO: 2021
MÊS: 02
DIA: 12

ANO: 2021
MÊS: 07
DIA: 08

ANO: 2021
MÊS: 07
DIA: 08

ANO: 2019
MÊS: 10
DIA: 28

ANO: 2021
MÊS: 07
DIA: 08

ANO: 2019
MÊS: 12
DIA: 18

ANO: 2019
MÊS: 09
DIA: 03

ANO: 2019
MÊS: 10
DIA: 28

ANO: 2019
MÊS: 09
DIA: 03

ANO: 2020
MÊS: 03
DIA: 11

ANO: 2019
MÊS: 10
DIA: 28

ANO: 2021
MÊS: 03
DIA: 29

ANO: 2019
MÊS: 10
DIA: 28

ANO: 2019
MÊS: 10
DIA: 28

ANO: 2019
MÊS: 09
DIA: 03

ANO: 2019
MÊS: 10
DIA: 28

ANO: 2021
MÊS: 07
DIA: 21

ANO: 2020
MÊS: 10
DIA: 29

ANO: 2021
MÊS: 07
DIA: 08

ANO: 2021
MÊS: 07
DIA: 21

ANO: 2021
MÊS: 03
DIA: 29

ANO: 2019
MÊS: 10
DIA: 28

ANO: 2021
MÊS: 07
DIA: 08

ANO: 2021
MÊS: 03
DIA: 29

ANO: 2021
MÊS: 07
DIA: 21



In [94]:
# selecionando data específica:
for i in dataframe.divulgacao_dia_publicacao:
    if i == 0:
        pass
    elif i[:4] == '2015' and i[5:7] == '07':
        print(i)
    

2015-07-21
2015-07-21
2015-07-07
2015-07-14
2015-07-21
2015-07-15
2015-07-21
2015-07-15
2015-07-07
2015-07-21
2015-07-21
2015-07-21
2015-07-15
2015-07-15
2015-07-21
2015-07-21
2015-07-21
2015-07-21
2015-07-21
2015-07-15
2015-07-27
2015-07-16


In [101]:
# usando condições:
# selecionando data específica:
for i in dataframe.divulgacao_dia_publicacao:
    if i == 0:
        pass
    elif i[:4] == '2015' and i[5:7] == '07' and int('17') <= int(i[8:]) <= int('30'): # dias entre 17 e 30
        print(i)
    

2015-07-21
2015-07-21
2015-07-21
2015-07-21
2015-07-21
2015-07-21
2015-07-21
2015-07-21
2015-07-21
2015-07-21
2015-07-21
2015-07-21
2015-07-27


In [174]:
# criar coluna com dia e horas juntos:
dataframe['hora_dia'] = dataframe.divulgacao_dia_publicacao.astype(str) + ' ' + dataframe.ocorrencia_hora.astype(str)
dataframe.head()
if '23/08/2021' < '28/08/2021':
    print('k')
#dataframe['data_hora'] = [[x for x in vetor][x for x in vetor2]]


k


In [166]:
dataframe.dtypes

codigo_ocorrencia1                float64
codigo_ocorrencia2                float64
codigo_ocorrencia3                float64
codigo_ocorrencia4                float64
ocorrencia_classificacao           object
ocorrencia_latitude                object
ocorrencia_longitude               object
ocorrencia_cidade                  object
ocorrencia_uf                      object
ocorrencia_pais                    object
ocorrencia_aerodromo               object
ocorrencia_dia                     object
ocorrencia_hora                    object
investigacao_aeronave_liberada     object
investigacao_status                object
divulgacao_relatorio_numero        object
divulgacao_relatorio_publicado     object
divulgacao_dia_publicacao          object
total_recomendacoes               float64
total_aeronaves_envolvidas        float64
ocorrencia_saida_pista,,           object
nova                               object
coluna                             object
hora_dia                          

In [196]:
# realizar agrupamentos:
dataframe.groupby(['ocorrencia_classificacao']).size()  # pega a quantidade
variavel = dataframe.groupby(['ocorrencia_uf']).size()
maior = []
for i in variavel:
    if i > 0:
        maior.append(i)
aq = sorted(maior)
